In [1]:
import pandas as pd

In [2]:
transaction = pd.read_csv('csv/transaction.csv', delimiter=';')
transaction.head()

,transaction_id,item_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,NaN,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,NaN,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"
2,3,NaN,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82"
3,4,NaN,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10"
4,5,NaN,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48"


In [3]:
item = pd.read_csv('csv/item.csv', delimiter=';')
item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   item_id        204 non-null    float64
 1   product_id     204 non-null    float64
 2   brand          203 non-null    object 
 3   product_line   203 non-null    object 
 4   product_class  203 non-null    object 
 5   product_size   203 non-null    object 
 6   standard_cost  203 non-null    object 
dtypes: float64(2), object(5)
memory usage: 11.3+ KB


In [4]:
# Excel не дочистил дубликаты в item. Дочищаем.
item_columns = ['product_id', 'brand', 'product_line', 'product_class', 'product_size', 'standard_cost']
upd_item = item.drop_duplicates(item_columns)
upd_item.info()

<class 'pandas.core.frame.DataFrame'>
Index: 203 entries, 0 to 204
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   item_id        202 non-null    float64
 1   product_id     202 non-null    float64
 2   brand          201 non-null    object 
 3   product_line   201 non-null    object 
 4   product_class  201 non-null    object 
 5   product_size   201 non-null    object 
 6   standard_cost  201 non-null    object 
dtypes: float64(2), object(5)
memory usage: 12.7+ KB


In [5]:
# сохраняем обновленный файл item
upd_item.to_csv('csv/upd_item.csv', index=False)

In [6]:
# проставляем в таблицу transaction соответствующие item_id из таблицы item
upd_trans = transaction.drop(columns='item_id')
# upd_trans = upd_trans.merge(item, how='left', on=item_columns)
upd_trans = upd_trans.merge(upd_item, how='left', on=item_columns)
upd_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    20000 non-null  int64  
 1   product_id        20000 non-null  int64  
 2   customer_id       20000 non-null  int64  
 3   transaction_date  20000 non-null  object 
 4   online_order      19640 non-null  object 
 5   order_status      20000 non-null  object 
 6   brand             19803 non-null  object 
 7   product_line      19803 non-null  object 
 8   product_class     19803 non-null  object 
 9   product_size      19803 non-null  object 
 10  list_price        20000 non-null  object 
 11  standard_cost     19803 non-null  object 
 12  item_id           20000 non-null  float64
dtypes: float64(1), int64(3), object(9)
memory usage: 2.0+ MB


In [7]:
# убираем уже не нужные столбцы, которые перенесли в таблицу item
upd_trans = upd_trans.drop(columns=item_columns)
# возвращаем столбец item_id на вторую позицию
upd_trans = upd_trans[['transaction_id', 'item_id', 'customer_id','transaction_date','online_order','order_status','list_price']] 
upd_trans.head()

,transaction_id,item_id,customer_id,transaction_date,online_order,order_status,list_price
0,1,30.0,2950,25.02.2017,False,Approved,"71,49"
1,2,32.0,3120,21.05.2017,True,Approved,"2091,47"
2,3,94.0,402,16.10.2017,False,Approved,"1793,43"
3,4,179.0,3135,31.08.2017,False,Approved,"1198,46"
4,5,161.0,787,01.10.2017,True,Approved,"1765,3"


In [8]:
# сохраняем обновленную таблицу транзакций в новый файл
upd_trans.to_csv('csv/upd_transaction.csv', index=False)

In [11]:
# Обработаем также файт customer. Уберем паразитную колонку и сконвертируем deceased_indicator и owns_car в bool
customer = pd.read_csv('csv/customer.csv', delimiter=';')
customer.head(1)

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,property_valuation,Unnamed: 13
0,1,Laraine,Medendorp,Female,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,10,NaN


In [17]:
upd_customer = customer.drop(columns='Unnamed: 13')
upd_customer.head(1)

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,property_valuation
0,1,Laraine,Medendorp,Female,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,10


In [19]:
upd_customer['deceased_indicator'] = upd_customer['deceased_indicator'].map(lambda x: 1 if x == 'Y' else 0)
upd_customer['deceased_indicator'].value_counts()


deceased_indicator
0    3998
1       2
Name: count, dtype: int64

In [21]:
upd_customer['owns_car'] = upd_customer['owns_car'].map(lambda x: True if x == 'Yes' else False)
upd_customer['owns_car'].value_counts()

owns_car
True     2024
False    1976
Name: count, dtype: int64

In [22]:
upd_customer.to_csv('csv/upd_customer.csv', index=False)